In [2]:
from Utils import load_pickle, save_pickle

vector = load_pickle('teste','/workspace/data/output/set12/high_noisy_25/results/array_nln_high_filtereds.pkl')

In [5]:
for v in vector:
    print(v['file_name'])


06.png
01.png
03.png
10.png
05.png
09.png
11.png
02.png
07.png
04.png
08.png
